In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models


def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):
    
    loss_history = []
    val_loss_history = []

    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_train_epoch = 0
        total_train_epoch = 0
        num_batches = 0
        model.train()
        for i, data in enumerate(train_loader):
            inputs, labels = data
            labels = labels.float().unsqueeze(1) 
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_train_epoch += (predicted == labels).sum().item()
            total_train_epoch += labels.size(0)    
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")
        loss_history.append(running_loss/len(train_loader))    




def test_model(model, test_loader):
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    correct = 0
    tot = 0
    y_t = []
    y_p = []
    
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            labels = labels.float().unsqueeze(1)
            outputs = model(inputs)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            tot += labels.size(0)
            correct += (predicted == labels).sum().item()

            y_t.extend(labels.numpy())
            y_p.extend(predicted.numpy())
    print(f"Accuracy_test: {100*correct / tot}")

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

model_ft = models.alexnet(pretrained=True)

# Do the things required for fine-tuning before training the model. Vi vill ju byta ut sista lagret och ta in det vi vill klassifiera

num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)  # 10 klasser

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

# Train the model
trained_model_ft = train_model(model_ft, criterion, optimizer, trainloader, epochs)

# Test the model
tested_model = test_model(model_ft, testloader)


# För feature extraction:

#for param in model_ft.features.parameters():
#    param.requires_grad = False

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Dataförberedelse
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # Skal om CIFAR-10 bilder från 32x32 till 224x224
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

def train_model(model, criterion, optimizer, train_loader, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)    

            # Skriv ut statistik var 100:e batch (eller anpassa efter behov)
            if (i + 1) % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct_train / total_train:.2f}%")
    
    torch.save(model.state_dict(), "best_model.pth")

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


model_ft = models.alexnet(pretrained=True)
num_ftrs = model_ft.classifier[6].in_features
model_ft.classifier[6] = nn.Linear(num_ftrs, 10)  # 10 klasser

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

train_model(model_ft, criterion, optimizer, trainloader, num_epochs=10)

model_ft.load_state_dict(torch.load("best_model.pth"))
test_model(model_ft, testloader)

# För Feature Extraction (frys feature-extraktorn)
# for param in model_ft.features.parameters():
#     param.requires_grad = False


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/10, Batch 100/782, Loss: 1.9135
Epoch 1/10, Batch 200/782, Loss: 1.7779
Epoch 1/10, Batch 300/782, Loss: 1.6913
Epoch 1/10, Batch 400/782, Loss: 1.6442
Epoch 1/10, Batch 500/782, Loss: 1.6064
Epoch 1/10, Batch 600/782, Loss: 1.3247
Epoch 1/10, Batch 700/782, Loss: 1.7545
Epoch 1/10, Average Loss: 1.6440, Accuracy: 38.71%
Epoch 2/10, Batch 100/782, Loss: 1.3039
Epoch 2/10, Batch 200/782, Loss: 1.3651
Epoch 2/10, Batch 300/782, Loss: 1.1943
Epoch 2/10, Batch 400/782, Loss: 1.3282
Epoch 2/10, Batch 500/782, Loss: 1.2627
Epoch 2/10, Batch 600/782, Loss: 1.1728
Epoch 2/10, Batch 700/782, Loss: 0.9610
Epoch 2/10, Average Loss: 1.1659, Accuracy: 58.08%
Epoch 3/10, Batch 100/782, Loss: 0.7340
Epoch 3/10, Batch 200/782, Loss: 0.8316
Epoch 3/10, Batch 300/782, Loss: 0.9665
Epoch 3/10, Batch 400/782, Loss: 0.8827
Epoch 3/10, Batch 500/782, Loss: 1.1845
Epoch 3/10, Batch 600/782, Loss: 1.2066
Epoch 3/10, Batch 700/

C:\Users\lindh\AppData\Local\Temp\ipykernel_13664\3898665678.py:81: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_ft.load_state_dict(torch.load("best_model.pth"))


Test Accuracy: 75.83%
